## Анализ пользовательского поведения в мобильном приложении

В данном проекте были изучены принципы событийной аналитики. Мы строим воронку продаж, исследуем путь пользователей до покупки. Проанализировали результаты A/B-теста введения новых шрифтов. Сравнили 2 контрольных группы между собой, убедились в правильном разделении трафика, а затем сравнили с тестовой группой

Выявлено, что новый шрифт значительно не повлияет на поведение пользователей.

На основе данных использования мобильного приложения для продажи продуктов питания проанализировали воронку продаж, а также оценили результаты A/B-тестирования

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [3]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [4]:
query = '''
WITH 
c AS (
SELECT user_id,   (CASE
                      WHEN nps_score <= 6 THEN "detractors"
                      WHEN nps_score > 6 and nps_score <= 8 THEN "passives"
                     WHEN nps_score > 8 THEN "promoters"
                 END) AS nps_group,
 
                  (CASE
                     WHEN gender_segment = 0 THEN "мужской"
                     WHEN gender_segment = 1 THEN "женский"
                 END) AS gender
           FROM user
            ),
 
i AS (
select us.user_id,  
us.lt_day,
us.lt_day <= 365 as is_new ,
us.age,
us.gender_segment,
us.os_name,
us.cpe_type_name,
lo.country,
lo.city,
ags.title as age_segment ,
ts.title as traffic_segment ,
ls.title as lifetime_segment 
,
us.nps_score,
us.nps_score 
from user as us
LEFT JOIN location AS lo  ON us.location_id=lo.location_id
LEFT JOIN age_segment AS ags  ON us.age_gr_id=ags.age_gr_id
LEFT JOIN traffic_segment AS ts  ON us.tr_gr_id=ts.tr_gr_id
LEFT JOIN lifetime_segment AS ls  ON us.lt_gr_id=ls.lt_gr_id
)
 
 
select i.user_id,  
i.lt_day,
i.is_new,
i.age,
c.gender,
i.os_name,
i.cpe_type_name,
i.country,
i.city,
i.age_segment,
i.traffic_segment,
i.lifetime_segment,
i.nps_score,
c.nps_group
 
 from c INNER JOIN i ON c.user_id = i.user_id
 
            '''

In [5]:
df = pd.read_sql(query, engine)
df.head()

,user_id,lt_day,is_new,age,gender,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,женский,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,0,53.0,мужской,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,0,57.0,мужской,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters
3,A004TB,4190,0,44.0,женский,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,promoters
4,A004XT,1163,0,24.0,мужской,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,promoters


In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Ссылка на pdf-файл с презентацией:

https://disk.yandex.ru/i/40iXWl2dWyosQA

In [7]:
#NPS
d = df.assign(total_nps=lambda row: row['nps_group'].map({'promoters': 1, 'passives': 0, 'detractors': -1}))\
  .pivot_table(index=['is_new', 'gender'], columns='age_segment', values='total_nps', aggfunc='mean')

display(d)

total_nps = df.assign(total_nps=lambda row: row['nps_group'].map({'promoters': 1, 'passives': 0, 'detractors': -1}))\
  ['total_nps'].mean()
print(f"Total_nps: {total_nps:.1%}")

age_segment     01 до 16  02 16-24  03 25-34  04 35-44  05 45-54  06 55-64  \
is_new gender                                                                
0      женский -0.030651  0.001719  0.084923  0.213887  0.327369  0.439379   
       мужской -0.150183 -0.027697  0.028052  0.158118  0.262021  0.358026   
1      женский  0.222222  0.216469  0.309779  0.420068  0.514749  0.587825   
       мужской  0.058140  0.193298  0.261561  0.373902  0.455968  0.495995   

age_segment      07 66 +    08 n/a  
is_new gender                       
0      женский  0.543689 -0.166667  
       мужской  0.435383 -0.555556  
1      женский  0.598240  1.000000  
       мужской  0.577167 -0.714286

Total_nps: 21.9%
